# For cross-lingual texts

# For English Texts

In [ ]:
#Required Libraries and package of python
import nltk
import gensim
from gensim.models import Word2Vec
import string
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from nltk.tokenize import sent_tokenize,word_tokenize
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy import spatial
import spacy
nlp=spacy.load('en_core_web_sm')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

import nltk 
from nltk.tokenize import LineTokenizer 
from nltk.stem import PorterStemmer     
tk = LineTokenizer()   
lm = WordNetLemmatizer() 
from nltk.corpus import wordnet as wn
import re
import string

In [ ]:
#Pretrained Word2Vec model. It was pretrained on Google News Corpus
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
#Stopwords
stop_words = set(stopwords.words('english'))
sw=stop_words

In [ ]:
#STS-2017 Dataset is used here.Here, we are taking the first sentence and second sentence into two list. 
fh=open('STS.input.track51.en-en.txt')
c=0
s1=[]
s2=[]
for line in fh:
    line=line.lower()
    c+=1
    line=line.rstrip()
    sentences=line.split('\t')
    s1.append(sentences[0])
    s2.append(sentences[1]) 
print(c)

# Maximum word-level Similarity(MWL_S)

In [ ]:
file=open('resultwordcommean1.txt','w')
ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i].split(' ')
    s1word=[]
    for word in ns1:
        for c in string.punctuation:
            word=word.replace(c,'')
        if word not in sw:
            s1word.append(lemmatizer.lemmatize(word))
    #print(s1word)
    s2word=[]
    ns2=s2[i].split(' ')
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            s2word.append(lemmatizer.lemmatize(word))
    #print(s2word)
    max_score=[]
    for word in s1word:
        if word in model.wv.vocab:
            score=[]
            for w in s2word:
                if w in model.wv.vocab:
                    score.append(model.similarity(word,w))
        max_score.append(max(score))
    #print(max_score)
    
    max_score2=[]
    for word in s2word:
        if word in model.wv.vocab:
            score=[]
            for w in s1word:
                if w in model.wv.vocab:
                    score.append(model.similarity(word,w))
        max_score2.append(max(score))
    #print(max_score2)
    
    count=0
    summ=0
    for val in max_score:
        summ+=val
        count+=1
    average=(summ*5)/count
    
    count1=0
    summ1=0
    for val in max_score2:
        summ1+=val
        count1+=1
    average1=(summ1*5)/count1
    
    aver=(average+average1)/2
    #print('i:',i+1,aver)
    a=str(round(aver,6))+'\n'
    file.writelines(a)
    i+=1
    
file.close()
data1=[]
data2=[]
f1=open('resultwordcommean1.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
#print(len(data1),len(data2))
#mse = np.square(np.subtract(data2,data1)).mean()
#print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Average Feature Vector based Similarity(V_avgS):

In [ ]:
file=open('result1.txt','w')
ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i].split(' ')
    sum1=np.zeros((300,), dtype="float32")
    for word in ns1:
        for c in string.punctuation:
                word=word.replace(c,"")
        #print(word)
        if word not in sw:
            if(word in model.wv.vocab): 
                sum1=np.add(sum1,model[word])
    average1=np.divide(sum1,len(ns1),dtype="float32")
            
    ns2=s2[i].split(' ')
    sum2=np.zeros((300,),dtype="float32")
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            if(word in model.wv.vocab):
                sum2=np.add(sum2,model[word])
    average2=np.divide(sum2,len(ns2),dtype="float32")
    sim = 1 - spatial.distance.cosine(average1,average2)
    sim=sim*5
    a=str(round(sim,6))+'\n'
    file.writelines(a)
    #print(i,'similarity',sim)
    i+=1 
file.close()

from sklearn.metrics import mean_squared_error

data1=[]
data2=[]
f1=open('result1.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
#print(len(data1),len(data2))
#mse = np.square(np.subtract(data2,data1)).mean()
#print('MSE:',mse)
#ms=mean_squared_error(data2,data1)
#print('MS:',ms)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Maximum word-level similarity of NP, VP (MWLR_Sim)

In [ ]:
file=open('resultNP_VP_compare12.txt','w')

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

def listtostr(phrase):
    sent=''
    for word in phrase:
        sent=sent+" "+str(word.strip())
    return sent

def result(s1,s2):
    s1word=[]
    s2word=[]
    for word in s1:
        if word not in sw:
            s1word.append(word)
    for word in s2:
        if word not in sw:
            s2word.append(word)
            
    #print(s1word,s2word)
    if len(s1word)==0 or len(s2word)==0:
        aver=0
    else:
        max_score=[]
        for word in s1word:
            score=[0]
            if word in model.wv.vocab:
                for w in s2word:
                    if w in model.wv.vocab:
                        score.append(model.similarity(word,w))
            max_score.append(max(score))
        #print(max_score)
    
        max_score2=[]
        for word in s2word:
            score=[0]
            if word in model.wv.vocab:
                for w in s1word:
                    if w in model.wv.vocab:
                        score.append(model.similarity(word,w))
            max_score2.append(max(score))
        #print(max_score2)
    
        average=sum(max_score)/(len(max_score))
    
        average1=sum(max_score2)/(len(max_score2))
    
        aver=(average+average1)/2
    return aver

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
    #print(ns1)
    npp,vpp=partitioning(ns1)
    #print(npp,vpp)
    sen1np=listtostr(npp).strip()
    sen1np=sen1np.split(' ')
    #print(sen1np, len(sen1np))
    sen1vp=listtostr(vpp).strip()
    sen1vp=sen1vp.split(' ')
        
    ns2=s2[i]
    #print(ns2)
    npp,vpp=partitioning(ns2)
    #print(npp,vpp)
    sen2np=listtostr(npp).strip()
    sen2np=sen2np.split(' ')
    sen2vp=listtostr(vpp).strip()
    sen2vp=sen2vp.split(' ')
    
    avernp=result(sen1np,sen2np)
    #print('NP:',avernp)
    avervp=result(sen1vp,sen2vp)
    #print('VP:',avervp)
    #if(avernp==0 or avervp==0):
     #   av=(avernp+avervp)*5
    #else:
    av=avernp*4+avervp
    av=str(round(av,6))+'\n'
    #print('No:',i+1,'similarity:',av)
    file.writelines(av)
    
    i=i+1
    
file.close()

data1=[]
data2=[]
f1=open('resultNP_VP_compare12.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse=np.square(np.subtract(data2,data1)).mean()
print("MSE:",mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Parts-of-Speech(POS) tagged-based similarity

In [ ]:
file=open('resultpos1.txt','w')

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i].split(' ')
    s1word=[]
    for word in ns1:
        for c in string.punctuation:
            word=word.replace(c,'')
        if word not in sw:
            s1word.append(word)
    #print(s1word)
    s2word=[]
    ns2=s2[i].split(' ')
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            s2word.append(word)
    #print(s2word)
    tag1=nltk.pos_tag(s1word,tagset='universal')
    tag2=nltk.pos_tag(s2word,tagset='universal')
    #print(tag1,tag2)
    k=0
    max_score=[]
    for k in tag1:
        sscore=[]
        for j in tag2:
            if (k[1]==j[1]):
                if k[0] in model.wv.vocab:
                    if j[0] in model.wv.vocab:
                        a=model.similarity(k[0],j[0])
                        #print(k[0],j[0],a)
                        sscore.append(a)
                    #print(sscore)
        if(len(sscore)>=1):
            max_score.append(max(sscore))
    #print(max_score)
    count=0
    summ=0
    for val in max_score:
        summ+=val
        count+=1
    average=(summ*5)/count
    #print('i:',i+1,average)
    a=str(round(average,6))+'\n'
    file.writelines(a)
    i+=1
file.close()

data1=[]
data2=[]
f1=open('resultpos1.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# For Bengali texts

In [ ]:
import nltk
import gensim
from gensim.models import KeyedVectors
from bnlp import BengaliWord2Vec
bwv = BengaliWord2Vec()
from bnlp import NLTKTokenizer
bnltk = NLTKTokenizer()
from bnlp import NER
bn_ner = NER()
from bnlp import POS
bn_pos = POS()
import fasttext
import numpy as np
from scipy import spatial
from scipy.stats import pearsonr
from scipy.stats import spearmanr

In [ ]:
from gensim.models.wrappers import FastText
model=FastText.load_fasttext_format('wiki.bn.bin')

In [ ]:
#STS-2017 Dataset is used here.Here, we are taking the first sentence and second sentence into two list. 
fh=open('STS-Bengali.txt',encoding = 'utf-8')
c=0
s1=[]
s2=[]
for line in fh:
    line=line.lower()
    c+=1
    line=line.rstrip()
    sentences=line.split('$')
    s1.append(sentences[0])
    s2.append(sentences[1]) 
print(c)

In [ ]:
punctuation=['।', ',', ':', ';', '-', '!','?']

In [ ]:
i=0
s11=[]
while i<len(s1):
    wd=bnltk.word_tokenize(s1[i])
    for word in wd:
        if word in punctuation:
            wd.remove(word)
    i=i+1
    s11.append(wd)

i=0
s22=[]
while i<len(s1):
    wd=bnltk.word_tokenize(s2[i])
    for word in wd:
        if word in punctuation:
            wd.remove(word)
    i=i+1
    s22.append(wd)

In [ ]:
from bangla_stemmer.stemmer import stemmer
stmr = stemmer.BanglaStemmer()

# Maximum word-level Similarity(MWL_S)

In [ ]:
file=open('Bengali_result_stemmer.txt','w')
i=0
similarity2=[]
while i<250:
    s11[i]=stmr.stem(s11[i])
    s22[i]=stmr.stem(s22[i])
    #print(s11[i])
    #print(s22[i])
    max_score=[]
    for word in s11[i]:
        if word in model.wv.vocab:
            score=[]
            for w in s22[i]:
                if w in model.wv.vocab:
                    score.append(model.similarity(word,w))
        max_score.append(max(score))
    #print(max_score)
    
    max_score2=[]
    for word in s22[i]:
        if word in model.wv.vocab:
            score=[]
            for w in s11[i]:
                if w in model.wv.vocab:
                    score.append(model.similarity(word,w))
        max_score2.append(max(score))
    #print(max_score2)
    
    count=0
    summ=0
    a=max(max_score)
    for val in max_score:
        if(val >= a/2):
            summ+=val
            count+=1
    average=(summ*5)/len(max_score)
    
    count1=0
    summ1=0
    b=max(max_score2)
    for val in max_score2:
        if(val>= b/2):
            summ1+=val
            count1+=1
    average1=(summ1*5)/len(max_score2)
    
    aver=(average+average1)/2
    #similarity4.append(aver)
    a=str(round(aver,6))+'\n'
    file.writelines(a)
    similarity2.append(aver)
    i=i+1
    #print('NO:',i,"sim:",aver)
file.close()


In [ ]:
data1=[]
data2=[]
f1=open('Bengali_result_stemmer.txt')
f2=open('Gold-Standard.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# BOW approach

In [ ]:
#For Bag of words and tf-idf
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(norm=None)

In [ ]:
f1=open('bengali_bow.txt','w')

def vectorize(tokens):
    vector=[]
    for w in vocab:
        vector.append(tokens.count(w))
    return vector

def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

i=0
while i<250:
    vocab=unique(s11[i]+s22[i])
    
    vector1=vectorize(s11[i])
    
    vector2=vectorize(s22[i])
    
    sim = 1 - spatial.distance.cosine(vector1,vector2)
    
    a=str(round(sim,2))+'\n'
    f1.writelines(a)
    
    i=i+1
f1.close() 

data1=[]
data2=[]
f1=open('Bengali_bow.txt')
f2=open('Gold-Standard.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
data1=listvalue(f1)
data2=listvalue(f2)
#print(len(data1),len(data2))
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Maximum word-level similarity of NP, VP (MWL_NVP_S)

In [ ]:
from bnlp import POS
bn_pos = POS()
model_path = "bn_pos.pkl"

In [ ]:
similarity3=[]
i=0
while i<250:
    n1=[]
    p1=[]
    aj1=[]
    v1=[]
    
    n2=[]
    p2=[]
    aj2=[]
    v2=[]
    res = bn_pos.tag(model_path, s1[i])
    res2=bn_pos.tag(model_path, s2[i])
    
    for word in res:
        if word[1]=='VM':
            v1.append(word[0])
        elif word[1]=='NC' or word[1]=='NP':
            n1.append(word[0])
        elif word[1]=='PPR' or word[1]=='PRF':
            p1.append(word[0])
        elif word[1]=='JJ' or word[1]=='JQ':
            #aj1.append(word[0])
            n1.append(word[0])
    
    for word in res2:
        if word[1]=='VM':
            v2.append(word[0])
        elif word[1]=='NC' or word[1]=='NP':
            n2.append(word[0])
        elif word[1]=='PPR' or word[1]=='PRF':
            p2.append(word[0])
        elif word[1]=='JJ' or word[1]=='JQ':
            #aj2.append(word[0])
            n2.append(word[0])
            
    def get_sim(p1,p2):
        p1=stmr.stem(p1)
        p2=stmr.stem(p2)
        
        max_score=[]
        max_score2=[]
        
        for word in p1:
            score=[]
            if word in model.wv.vocab:
                
                for w in p2:
                    if w in model.wv.vocab:
                        score.append(model.similarity(word,w))
            if len(score)!=0:
                max_score.append(max(score))
    #print(max_score)
    
        max_score2=[]
        for word in p2:
            score=[]
            if word in model.wv.vocab:
                
                for w in p1:
                    if w in model.wv.vocab:
                        score.append(model.similarity(word,w))
                
            if len(score)!=0:
                max_score2.append(max(score))
        if len(max_score)!=0:
            sim=sum(max_score)/len(max_score)
        else:
            sim=0
        if len(max_score2)!=0:
            sim2=sum(max_score)/len(max_score2)
        else:
            sim2=0
        similar=(sim+sim2)/2
    
        return similar
    
    nsim=get_sim(n1,n2)
    vsim=get_sim(v1,v2)
    ajsim=get_sim(aj1,aj2)
    psim=get_sim(p1,p2)
    
    sim_score=nsim*4.25 +0.75* vsim
    similarity3.append(sim_score)
    i=i+1

In [ ]:
data1=[]
data2=[]

f2=open('Gold-Standard.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value

data2=listvalue(f2)
print(len(similarity3),len(data2))
corr,_=pearsonr(similarity3,data2)
scor,_=spearmanr(similarity3,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Parts-of-Speech(POS) tagged-based similarity

In [ ]:
punctuation=['।', ',', ':', ';', '-', '!','?']

In [ ]:
similarity=[]
i=0
while i<250:
    text = []
    text2= []
    res = bn_pos.tag(model_path, s1[i])
    res2=bn_pos.tag(model_path, s2[i])
    k=0
    while(k<len(res)):
        word=res[k][0]
        if word in punctuation:
            k=k+1
        else:
            word=stmr.stem(word)
            score=[]
            if word in model.wv.vocab:
            #score=[]
                j=0
                while(j<len(res2)):
                    if(res[k][1]==res2[j][1]):
                        word2=res2[j][0]
                        word2=stmr.stem(word2)
                        if word2 in model.wv.vocab:
                            #print(word, word2)
                            score.append(model.similarity(word,word2))
                    j=j+1
                if len(score)!=0:
                    text.append(max(score))
            k=k+1
    if len(text)!=0:
        sim=sum(text)/len(text)
    #print(text)
    #print("similarity:",sim*5)
    
    k=0
    while(k<len(res2)):
        word=res2[k][0]
        if word in punctuation:
            k=k+1
        else:
            word=stmr.stem(word)
            score2=[]
            if word in model.wv.vocab:
                #score2=[]
                j=0
                while(j<len(res)):
                    if(res2[k][1]==res[j][1]):
                        word2=res[j][0]
                        word2=stmr.stem(word2)
                        if word2 in model.wv.vocab:
                            #print(word, word2)
                            score2.append(model.similarity(word,word2))
                    j=j+1
                if len(score2)!=0:
                    text2.append(max(score2))
            k=k+1
    if len(text2)!=0:
        sim2=sum(text2)/len(text2)
    #print(text2)
    #print(sim2*5)
    average=(sim+sim2)/2
    
    similarity.append(average*5)
    print('no:',i+1, average*5)
    i=i+1

In [ ]:
data1=[]
data2=[]
#f1=open('result1.txt')
f2=open('Gold-Standard.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line))
    return value
#data1=listvalue(f1)
data2=listvalue(f2)
print(len(similarity),len(data2))
corr,_=pearsonr(similarity,data2)
scor,_=spearmanr(similarity,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Average Feature Vector based Similarity(V_avgS):

In [ ]:
#file=open('result1.txt','w')
ns1=[]
ns2=[]
i=0
similarity2=[]
similarity3=[]
print('Length:',len(s1))

while(i<len(s11)):
    #print(s11[i])
    sum1=np.zeros((300,), dtype="float32")
    c=0
    for word in s11[i]:
        if(word in model.wv.vocab):
            c=c+1
            #print(word)
            sum1=np.add(sum1,model[word])
    #average1=np.divide(sum1,len(s11[i]),dtype="float32")
    average1=np.divide(sum1,c,dtype="float32")
   
    #print(s22[i])
    sum2=np.zeros((300,),dtype="float32")
    c=0
    for word in s22[i]:
        if(word in model.wv.vocab):
            c=c+1
            #print(word)
            sum2=np.add(sum2,model[word])
    average2=np.divide(sum2,c,dtype="float32")
    
    sim2 = 1 - spatial.distance.cosine(sum1,sum2)
    sim2=sim2*5
    similarity3.append(sim2)
    sim = 1 - spatial.distance.cosine(average1,average2)
    sim=sim*5
    
    similarity2.append(sim)
    #print(i, sim)
    #a=str(round(sim,6))+'\n'
    #file.writelines(a)
    #print(i,'similarity',sim)
    i+=1 
#file.close()


In [ ]:

data1=[]
data2=[]
#f1=open('result1.txt')
f2=open('Gold-Standard.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
#data1=listvalue(f1)
data2=listvalue(f2)
#print(len(similarity2),len(data2))
print('average\n')
corr,_=pearsonr(similarity2,data2)
scor,_=spearmanr(similarity2,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)
#print('sum\n')
#print(len(similarity3),len(data2))
corr,_=pearsonr(similarity3,data2)
scor,_=spearmanr(similarity3,data2)
#print('Pearson:',corr*100,'Spearman:',scor*100)

# Monolingual Texts (English)

In [ ]:
import nltk
import gensim
from gensim.models import Word2Vec
import string
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from nltk.tokenize import sent_tokenize,word_tokenize
import numpy as np
from scipy import spatial

filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
fh=open('STS.input.track5.en-en.txt')
c=0
s1=[]
s2=[]
for line in fh:
    line=line.lower()
    c+=1
    line=line.rstrip()
    sentences=sent_tokenize(line)
    s1.append(sentences[0])
    s2.append(sentences[1])    
#print(s1,s2)
print(c)

# Maximum-Role level similarity(RM_sim):

In [ ]:
file=open('newNpVp.txt','w')

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

def vectorvalue(subsent):
    #print(subsent)
    subsent=subsent.split(" ")
    sum1=np.zeros((300,), dtype="float32")
    for word in subsent:
        if word not in sw:
            if(word in model.wv.vocab):
                sum1=np.add(sum1,model[word])
    if len(subsent)>0:
        average1=np.divide(sum1,len(subsent),dtype="float32")
    else:
        average1=sum1
    return average1

def listvalue(pp):
    pplist=[]
    if(len(pp)>0):
        j=0
        while(j<len(pp)):
            pplist.append(vectorvalue(pp[j]))
            j=j+1
    else:
        pplist.append(np.zeros((300,), dtype="float32"))
    return pplist

def score(nplist,np2list,vplist,vp2list,a):
    simlist=[]
    if(np.all(nplist)==0 and np.all(np2list)==0 and np.all(vplist)==0 and np.all(vp2list)==0):
        aa=0
    else:
        for value in nplist:
            npsim=[]
            for value2 in np2list:
                sim=(1 - spatial.distance.cosine(value,value2))*(5/a)
                if( np.isnan(sim)==1):
                    sim=0
                npsim.append(sim)
            #print('np:',max(npsim))
            simlist.append(max(npsim))
        
        for v in vplist:
            vpsim=[]
            for v2 in vp2list:
                sim=(1-spatial.distance.cosine(v,v2))*(5/a)
                if np.isnan(sim)==1:
                    sim=0
                vpsim.append(sim)
            #print('vp:',max(vpsim))
            simlist.append(max(vpsim))
        if(len(simlist)>0):    
            bb=max(simlist)
            p=0
            while(p<len(simlist)):
                if simlist[p]<bb/2:
                    simlist[p]=0
                p=p+1
        aa = sum(simlist)
    return aa

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
    #print(ns1)
    nplist=[]
    vplist=[]
    np2list=[]
    vp2list=[]
    npp,vpp=partitioning(ns1)
    #print(npp,vpp)
    nplist=listvalue(npp)
    vplist=listvalue(vpp)
    a=len(npp)+len(vpp)
    
    ns2=s2[i]
    #print(ns2)
    npp2,vpp2=partitioning(ns2)
    #print(npp2,vpp2)
    np2list=listvalue(npp2)
    vp2list=listvalue(vpp2)
    b=len(npp2)+len(vpp2)
    
    if a>=b:
        sim=score(nplist,np2list,vplist,vp2list,a)
    else:
        sim=score(np2list,nplist,vp2list,vplist,b)
    #print(i+1,'similarity:',sim)
    sim=str(round(sim,2))+'\n'
    file.writelines(sim)
    i=i+1
    
file.close()

data1=[]
data2=[]
f2=open('STS.gs.track5.en-en.txt')
f1=open('newNpVp.txt')
#f2=open('newNpVp.txt')

def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
#print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Feature Threshold based similiarity (Fthre_S):

In [ ]:
file=open('new_vector_form.txt','w')
ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<250):
    ns1=s1[i].split(' ')
    sum1=np.zeros((300,), dtype="float32")
    w=0
    for word in ns1:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            if(word in model.wv.vocab):
                if w==0:
                    sum1=np.add(sum1,model[word])
                    w=w+1
                else:
                    av=sum(sum1)/300
                    a=list(model[word])
                    sum1=list(sum1)
                    k=0
                    while(k<300):
                        if abs(a[k]-sum1[k])>=0.135: # works better at 0.135
                            sum1[k]=a[k]+sum1[k]
                            k=k+1
                        else:
                            sum1[k]=sum1[k]
                            k=k+1
                    sum1=np.asarray(sum1)
    average1=np.divide(sum1,1,dtype="float32")
            
    ns2=s2[i].split(' ')
    sum2=np.zeros((300,),dtype="float32")
    
    w=0
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            if(word in model.wv.vocab):
                if w==0:
                    sum2=np.add(sum2,model[word])
                    w=w+1
                else:
                    av=sum(sum2)/300
                    a=list(model[word])
                    sum2=list(sum2)
                    k=0
                    while(k<300):
                        if abs(a[k]-sum2[k])>=0.135: # works better at 0.135
                            sum2[k]=a[k]+sum2[k]
                            k=k+1
                        else:
                            sum2[k]=sum2[k]
                            k=k+1
                    sum2=np.asarray(sum2)        
    average2=np.divide(sum2,1,dtype="float32")

    sim = 1 - spatial.distance.cosine(average1,average2)
    sim=sim*5
    a=str(round(sim,6))+'\n'
    file.writelines(a)
    #print(i,'similarity',sim)
    i+=1
file.close()

In [ ]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import numpy as np
from sklearn.metrics import mean_squared_error

data1=[]
data2=[]
f1=open('new_vector_form.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
#print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
#print('MSE:',mse)
ms=mean_squared_error(data2,data1)
#print('MS:',ms)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

# Feature Vector based (AVF_Sim)

In [ ]:
file=open('resultNP_VP12.txt','w')

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

def vectorvalue(subsent):
    #print(subsent)
    subsent=subsent.split(" ")
    sum1=np.zeros((300,), dtype="float32")
    for word in subsent:
        if word not in sw:
            if(word in model.wv.vocab):
                sum1=np.add(sum1,model[word])
    if len(subsent)>0:
        average1=np.divide(sum1,len(subsent),dtype="float32")
    else:
        average1=sum1
    return average1

def listtostr(phrase):
    sent=''
    for word in phrase:
        sent=sent+" "+str(word)
    return vectorvalue(sent)

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
    #print(ns1)
    npp,vpp=partitioning(ns1)
    #print(npp,vpp)
    sen1np=listtostr(npp)
    sen1vp=listtostr(vpp)
        
    ns2=s2[i]
    #print(ns2)
    npp,vpp=partitioning(ns2)
    #print(npp,vpp)
    sen2np=listtostr(npp)
    sen2vp=listtostr(vpp)
        
    if ((np.all(sen1np==0)) or (np.all(sen2np==0))):
            nppsim=0
    else:
        nppsim= 1 - spatial.distance.cosine(sen1np,sen2np)
    #print('nounphase:',nppsim)
    if ((np.all(sen1vp==0)) or (np.all(sen2vp==0))):
            vppsim=0
    else:
        vppsim=1 - spatial.distance.cosine(sen1vp,sen2vp)
    #print("verbphase:",vppsim)
    #if(nppsim==0 or vppsim==0):
       # similarity=(nppsim+vppsim)*5
    #elif(nppsim !=0 and vppsim !=0):
    similarity=nppsim*4+vppsim
    #print("No:",i,similarity)
    sim=str(round(similarity,2))+'\n'
    file.writelines(sim)
    i+=1
    
file.close()

data1=[]
data2=[]
f1=open('resultNP_VP12.txt')
f2=open('STS.gs.track5.en-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)
#P=67.51, S=70.18